In [7]:
import cv2
import numpy as np
import pandas as pd
import math
import os
import tensorflow as tf


# טוען את המודל movenet
interpreter = tf.lite.Interpreter(model_path="thunder3.tflite")
interpreter.allocate_tensors()

# פונקציה לחישוב זווית בין שלוש נקודות
def calculate_angle(p1, p2, p3):
    v1 = np.array(p1) - np.array(p2)
    v2 = np.array(p3) - np.array(p2)
    angle = np.arccos(
        np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-6)
    )
    return np.degrees(angle)

# נתיב לתיקיית הסרטונים
video_folder = "final_videos"
video_files = sorted([f for f in os.listdir(video_folder) if f.endswith(".mp4")])

dataset = []

for video_index, video_file in enumerate(video_files):
    video_path = os.path.join(video_folder, video_file)
    cap = cv2.VideoCapture(video_path)

    prev_keypoints = None
    prev_velocity = None
    prev_time = None

    frame_index = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        input_tensor = cv2.resize(frame, (256, 256))
        input_tensor = np.expand_dims(input_tensor.astype(np.float32), axis=0)

        interpreter.set_tensor(interpreter.get_input_details()[0]['index'], input_tensor)
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

        keypoints = keypoints_with_scores[0, 0, :, :2]  # x, y נורמליזציה
        scores = keypoints_with_scores[0, 0, :, 2]  # ציון ביטחון

        height, width, _ = frame.shape
        keypoints = keypoints * np.array([width, height])  # שינוי לגודל המקורי של התמונה

        current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0  # זמן נוכחי בשניות

        # חישוב מהירות ותאוצה
        velocity = None
        acceleration = None

        if prev_keypoints is not None and prev_time is not None:
            time_diff = current_time - prev_time

            if time_diff > 0:
                velocity = (keypoints - prev_keypoints) / time_diff
                if prev_velocity is not None:
                    acceleration = (velocity - prev_velocity) / time_diff

        # חישוב זוויות ברגליים ובמרפקים
        angles = {
            13: ("left_knee_angle", calculate_angle(keypoints[11], keypoints[13], keypoints[15])),  # ירך-ברך-קרסול שמאל
            14: ("right_knee_angle", calculate_angle(keypoints[12], keypoints[14], keypoints[16])),  # ירך-ברך-קרסול ימין
            7: ("left_elbow_angle", calculate_angle(keypoints[5], keypoints[7], keypoints[9])),  # כתף-מרפק-שורש כף יד שמאל
            8: ("right_elbow_angle", calculate_angle(keypoints[6], keypoints[8], keypoints[10]))  # כתף-מרפק-שורש כף יד ימין
        }

        # שמירת נתונים
        for i, (point, score) in enumerate(zip(keypoints, scores)):
            data_entry = {
                "video_id": video_index + 1,  # מזהה סרטון
                "frame": frame_index,
                "keypoint_id": i,
                "x": point[0],
                "y": point[1],
                "confidence": score,
                "velocity_x": velocity[i][0] if velocity is not None else None,
                "velocity_y": velocity[i][1] if velocity is not None else None,
                "acceleration_x": acceleration[i][0] if acceleration is not None else None,
                "acceleration_y": acceleration[i][1] if acceleration is not None else None
            }

            # הוספת זוויות רלוונטיות
            if i in angles:
                angle_name, angle_value = angles[i]
                data_entry[angle_name] = angle_value

            dataset.append(data_entry)

        prev_keypoints = keypoints.copy()
        prev_velocity = velocity.copy() if velocity is not None else None
        prev_time = current_time
        frame_index += 1

    cap.release()

# שמירת הנתונים לקובץ CSV
df = pd.DataFrame(dataset)
df.to_csv("motion_dataset.csv", index=False)

print("✅ סיום עיבוד הסרטונים, הנתונים נשמרו motion_dataset.csv")

✅ סיום עיבוד הסרטונים, הנתונים נשמרו motion_dataset.csv
